In [0]:
# Import necessary functions and types
from pyspark.sql import functions as F
from pyspark.sql.types import *
from delta.tables import DeltaTable
from pyspark.sql.functions import monotonically_increasing_id
from pyspark.sql.window import Window
from pyspark.sql.functions import col, count
from pyspark.sql.functions import row_number, lit, current_timestamp
from pyspark.sql.window import Window
from delta.tables import DeltaTable
from pyspark.sql.functions import coalesce, max, lit, col, current_timestamp
from pyspark.sql.window import Window
from delta.tables import DeltaTable
from pyspark.sql.functions import coalesce, max, lit, row_number, col, current_timestamp
from pyspark.sql.window import Window
from delta.tables import DeltaTable

In [0]:
silver_path = "/mnt/mock_prajwal/Mock2/silver/"
gold_path = "/mnt/mock_prajwal/Mock2/gold/"

In [0]:
df = spark.read.format("delta").load(silver_path + "FF_Customer_Details_Day0")
df.printSchema()

In [0]:
product_dim = spark.read.format("delta").load("/mnt/mock_prajwal/Mock2/gold/Dim_Product")
customer_dim = spark.read.format("delta").load("/mnt/mock_prajwal/Mock2/gold/Dim_customer")
sales_dim = spark.read.format("delta").load("/mnt/mock_prajwal/Mock2/gold/Dim_Sales_rep")

In [0]:
fact_df = df.join(customer_dim.select("customer_id", "customer_key"), on="customer_id", how="left").join(product_dim.select("product_name", "product_key"), on="product_name", how="left").join(sales_dim.select("sales_rep_id", "sales_rep_key"), on="sales_rep_id", how="left")

display(fact_df)

In [0]:
fact_df = fact_df.select(
    "order_id",
    "order_date",
    "ship_date",
    "profit",
    "quantity_ordered_new",
    "sales",
    "discount",
    "unit_price",
    "shipping_cost",
    "product_base_margin",
    "customer_id",
    "product_name",
    "sales_rep_id"
)

display(fact_df)

In [0]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType, DateType

schema = StructType([
    StructField("order_id", IntegerType(), True),
    StructField("order_date", StringType(), True),
    StructField("ship_date", DateType(), True),
    StructField("profit", DoubleType(), True),
    StructField("quantity_ordered_new", IntegerType(), True),
    StructField("sales", DoubleType(), True),
    StructField("discount", DoubleType(), True),
    StructField("unit_price", DoubleType(), True),
    StructField("shipping_cost", DoubleType(), True),
    StructField("product_base_margin", DoubleType(), True),
    StructField("customer_id", IntegerType(), True),
    StructField("product_name", StringType(), True),
    StructField("sales_rep_id", IntegerType(), True)
])

# Create the table if it does not exist
spark.sql("""
    CREATE TABLE IF NOT EXISTS Prajwal_Mock.Fact_table (
        order_id INT,
        order_date STRING,
        ship_date DATE,
        profit DOUBLE,
        quantity_ordered_new INT,
        sales DOUBLE,
        discount DOUBLE,
        unit_price DOUBLE,
        shipping_cost DOUBLE,
        product_base_margin DOUBLE,
        customer_id INT,
        product_name STRING,
        sales_rep_id INT
    )
    USING DELTA 
    LOCATION "/mnt/mock_prajwal/Mock2/gold/Fact"
""")

In [0]:
fact_df.write.format("delta").mode("append").option("mergeSchema", "true").save("/mnt/mock_prajwal/Mock2/gold/Fact")

In [0]:
# Count records in the gold layer
gold_count = fact_df.count()

# Assuming silver layer is stored in a DataFrame named silver_df
silver_df = spark.read.format("delta").load(silver_path + "FF_Customer_Details_Day0")
silver_count = silver_df.count()

# Display counts
display(spark.createDataFrame([(silver_count, gold_count)], ["Silver Layer Count", "Gold Layer Count"]))